In [7]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [21]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.3 MB/s eta 0:00:00


## --------------------------------------
## Primeira Parte - Traduzindo documentos
## --------------------------------------

In [22]:
import requests
from docx import  Document
import os


In [ ]:
subscription_key = 'DxxxxxxxGDUAb'
endpoint = 'https://api.cognitive.microsofttranslator.com/'
location = 'eastus'
language_destination = 'pt-br'


In [63]:
def translator_text(text, target_language):
	path = '/translate'
	constructed_url = endpoint + path
	headers = {
	'Ocp-Apim-Subscription-Key': subscription_key,
	'Ocp-Apim-Subscription-Region': location,
	'Content-type': 'application/json',
	'X-ClientTraceId': str(os.urandom(16))

	}

	body = [{'text': text}]
	params = {
		'api-version': '3.0',
		'from': 'en',
		'to': target_language
	}
	request = requests.post(constructed_url, params=params, headers=headers, json=body)
	response = request.json()
	return response[0]['translations'][0]['text']

In [64]:
translator_text("I wanna go home", language_destination)

'Eu quero ir para casa'

In [65]:
def translate_document(path):
	document = Document(path)
	full_text = []
	for paragraph in document.paragraphs:
		translated_text =  translator_text(paragraph.text, language_destination)
		full_text.append(translated_text)

	translated_doc = Document()
	for line in full_text:
		translated_doc.add_paragraph(line)
	path_translated = path.replace(".docx", f"{language_destination}.docx")
	translated_doc.save(path_translated)
	return path_translated

In [66]:
input_file = "/content/exemplo_musica.docx"
translate_document(input_file)

'/content/exemplo_musicapt-br.docx'

## --------------------------------
## Segunda Parte - Traduzindo URL
## --------------------------------

In [1]:
import requests
from bs4 import BeautifulSoup
from langchain_openai.chat_models.azure import AzureChatOpenAI

In [2]:
def extract_text_from_url(url):
	response = requests.get(url)
	soup = BeautifulSoup(response.text, 'html.parser')
	text = soup.get_text()
	return text

	if response.status_code == 200:
		soup = BeautifulSoup(response.text, 'html.parser')
		for script_or_style in soup(['script', 'style']):
			script_or_style.decompose()
		texto = soup.get_text(separator = ' ')

		linhas = (line.strip() for line in texto.splitlines())
		parts = (phrase.strip() for line in linhas for phrase in line.split(" "))
		texto_limpo = '\n'.join(part for part in parts if part)
	else:
		print(f"Failes to fetch the URL. Status code: {response.status_code}")
		return None

	text = soup.get_text()
	return text

extract_text_from_url('https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo')

"\n\n\n\nAzure Open AI in VNet - DEV Community\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n              Forem Feed\n            \n\n              Follow new Subforems to improve your feed\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDEV Community\n\nFollow\n\n            A space to discuss and keep up software development and manage your software career\n          \n\n\n\n\n\n\n\n\n\n\n\nGamers Forem\n\nFollow\n\n            An inclusive community for gaming enthusiasts\n          \n\n\n\n\n\n\n\n\n\n\n\nFuture\n\nFollow\n\n            News and discussion of science and technology such as AI, VR, cryptocurrency, quantum computing, and more.\n          \n\n\n\n\n\n\n\n\n\n\n\nMusic Forem\n\nFollow\n\n            From composing and gigging to gear, hot music takes, and everything in between.\n          \n\n\n\n\n\n\n\n\n\n\n\nVibe Coding Forem\n\nFollow\n\n            Discussing AI software developm

In [5]:
client = AzureChatOpenAI(
  azure_endpoint = 'https://blabla.openai.azure.com',
  api_key = '3xxxxx5xx5b4cxxx9180xxxxxxxx48b' ,
  api_version = "2025-01-01-preview",
  deployment_name = 'gpt-4.1-mini',
  max_retries = 0
)

In [14]:
def translate_article(text, lang):

	messages = [
    {"role": "system", "content": "Você atua como tradutor de textos."},
    {"role": "user", "content": f"Traduza o texto a seguir para o idioma {lang} e responda em markdown:\n\n{text}"}
]


	response = client.invoke(messages)
	print(response.content)

	return response.content

In [15]:
translate_article("Let's see if the deployment was succeeded.", "portugues")

Vamos ver se o implante foi bem-sucedido.


'Vamos ver se o implante foi bem-sucedido.'

In [18]:
url = 'https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo'
text = extract_text_from_url(url)
article = translate_article(text, "portugues")
print(article)

```markdown
# Azure Open AI em VNet - DEV Community

## Kenichiro Nakamura  
Publicado em 12 de outubro de 2023

---

Os modelos GPT estão hospedados em múltiplos provedores de serviço atualmente, e o Microsoft Azure é um deles.  
Apesar dos modelos em si serem os mesmos, existem muitas diferenças, incluindo:

- custo  
- funcionalidades  
- tipos de modelos e versões  
- localização geográfica  
- segurança  
- suporte  
- etc.

Um dos aspectos mais importantes ao usá-los em um ambiente empresarial é, claro, a **segurança**.  
Usando os recursos de segurança de rede do Azure com o Azure Open AI, os clientes podem consumir o serviço Open AI a partir da VNet, ou seja, nenhuma informação transita pela internet pública.

---

## Implantação de Exemplo

O repositório de exemplo do Azure fornece arquivos bicep para implantar o Azure Open AI dentro de um ambiente VNet.  
GitHub: [openai-enterprise-iac](https://github.com/Azure-Samples/openai-enterprise-iac)  

As principais funcionalidades u